In [28]:

from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
import boto3
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
import os
import random
from joblib import Parallel , delayed

In [29]:
wr.config.s3_endpoint_url = "http://localhost:8333"


In [30]:
vehicleType = "Saloon - 4 Dr"

In [31]:
partlistDf = wr.s3.read_parquet(
        f"s3://multilabel_df", dataset=True
    )


In [32]:
localFilesDf = wr.s3.read_csv(
        f"s3://imgs_labels/{vehicleType}_rear_view_right_img_labels.csv"
    )
localCaseId = localFilesDf["CaseID"].unique().tolist()

In [33]:
vTypeDf = partlistDf[(partlistDf["Vehicle_Type"] == vehicleType) & (partlistDf["CaseID"].isin(localCaseId))]
vTypeDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_engine',
       'vision_grille', 'vision_headlamp_lh', 'vision_headlamp_rh',
       'vision_door_front_lh', 'vision_door_front_rh', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'vision_windscreen_rear', 'bonnet',
       'bumper_front', 'air_conditioning', 'cooling_fan', 'grille',
       'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh',
       'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel',
       'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh',
       

In [36]:
minorityLabels = ["windscreen_rear", "rear_quarter_rh", "rear_panel", "rear_quarter_lh", "tail_lamp_rh", "tail_lamp_lh", "rear_compartment", "windscreen_front", "fender_front_lh", "fender_front_rh"]

In [37]:
for i in minorityLabels:
    minorityDf = vTypeDf[i].value_counts().reset_index()
    print(minorityDf)

   index  windscreen_rear
0      0             9633
1      1              337
   index  rear_quarter_rh
0      0             9345
1      1              625
   index  rear_panel
0      0        8217
1      1        1753
   index  rear_quarter_lh
0      0             9446
1      1              524
   index  tail_lamp_rh
0      0          8500
1      1          1470
   index  tail_lamp_lh
0      0          8558
1      1          1412
   index  rear_compartment
0      0              8389
1      1              1581
   index  windscreen_front
0      0              8933
1      1              1037
   index  fender_front_lh
0      0             7810
1      1             2160
   index  fender_front_rh
0      0             7701
1      1             2269
